In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, GRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

Using TensorFlow backend.


(159571, 6)
(159571, 150) (153164, 150)
mcgeddon 15239
clean 1471
yanking 56920
palatines 125209
rulebreaker 180184
correlating 46296
2196007
Preparing embedding matrix
Null word embeddings: 3388


In [2]:
from sklearn.metrics import log_loss,accuracy_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [3]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_model()
        batch_size = 64
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), 
                  callbacks=callbacks_list)
        
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [4]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

train_pred,test_pred = kf_train(fold_cnt=4,rnd=4)
print(train_pred.shape,test_pred.shape)    


# 40000, 150
# final 0.040685252623 0.984164833627, pub 9845

Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 489s 4ms/step - loss: 0.0579 - acc: 0.9800 - val_loss: 0.0458 - val_acc: 0.9824
Epoch 2/8
119678/119678 [==============================] - 490s 4ms/step - loss: 0.0438 - acc: 0.9833 - val_loss: 0.0424 - val_acc: 0.9836
Epoch 3/8
119678/119678 [==============================] - 485s 4ms/step - loss: 0.0410 - acc: 0.9842 - val_loss: 0.0421 - val_acc: 0.9832
Epoch 4/8
119678/119678 [==============================] - 487s 4ms/step - loss: 0.0389 - acc: 0.9849 - val_loss: 0.0413 - val_acc: 0.9837
Epoch 5/8
119678/119678 [==============================] - 488s 4ms/step - loss: 0.0372 - acc: 0.9855 - val_loss: 0.0406 - val_acc: 0.9839
Epoch 6/8
119678/119678 [==============================] - 489s 4ms/step - loss: 0.0356 - acc: 0.9861 - val_loss: 0.0406 - val_acc: 0.9839
Epoch 7/8
119678/119678 [==============================] - 491s 4ms/step - loss: 0.0339 - acc: 0.9867 - val_loss: 0

Epoch 4/8
119679/119679 [==============================] - 484s 4ms/step - loss: 0.0376 - acc: 0.9852 - val_loss: 0.0410 - val_acc: 0.9844
Epoch 5/8
119679/119679 [==============================] - 484s 4ms/step - loss: 0.0357 - acc: 0.9860 - val_loss: 0.0412 - val_acc: 0.9843
Epoch 6/8
119679/119679 [==============================] - 485s 4ms/step - loss: 0.0341 - acc: 0.9865 - val_loss: 0.0418 - val_acc: 0.9839
Epoch 7/8
119679/119679 [==============================] - 486s 4ms/step - loss: 0.0327 - acc: 0.9869 - val_loss: 0.0421 - val_acc: 0.9840
Epoch 8/8
119679/119679 [==============================] - 489s 4ms/step - loss: 0.0313 - acc: 0.9876 - val_loss: 0.0432 - val_acc: 0.9838
-------------------------------
0 0.0894374356979 0.965921126019
1 0.021876810766 0.990549661279
2 0.0451803694333 0.982634689261
3 0.00841699935096 0.997317808374
4 0.0594638434591 0.975709872095
5 0.0197360570308 0.992855844734
final 0.040685252623 0.984164833627
all eval None
(159571, 6) (153164, 6)


FileNotFoundError: [Errno 2] No such file or directory: "../results/pool_gru1_glove_sample_<_io.TextIOWrapper name='../glove.840B.300d.txt' mode='r' encoding='UTF-8'>.gz"

In [5]:
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/pool_gru1_glove_sample_4.gz", index=False, compression='gzip')
with open('../features/pool_gru_glove_4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.996088      0.612122  0.979455  0.129746  0.948955   
1  0000247867823ef7  0.000551      0.000014  0.000254  0.000006  0.000210   
2  00013b17ad220c46  0.001690      0.000379  0.002866  0.000068  0.000768   
3  00017563c3f7919a  0.000229      0.000010  0.000132  0.000032  0.000126   
4  00017695ad8997eb  0.007934      0.000175  0.000948  0.000172  0.000653   

   identity_hate  
0       0.663246  
1       0.000029  
2       0.000111  
3       0.000009  
4       0.000080  
